# Model Predictive Control Project

The aim of this project is to design a model predictive controller(MPC) to drive a vehicle in the simulator. MPC controller is an optimal controller and it is different from a PID controller. A PID controller calculates the acutator commands based on the propotional, integral and derivative operations of the error signal. Whereas a MPC controller predicts the system outputs for a finite number of steps in future and calculates the acutator commands by minimizing a cost function (function of error, usage of actuator etc.). MPC algorithm takes into account the actuator contrains and time delays.With mpc it is possible to acheive higher system performance solve more complex problems. The disadvantages of MPC are high computation intensive and it is harder to define the stability of the system with MPC.

The following picture shows the overview of the MPC.

![MPC](./Pictures/MPC.png)

The following points shows the steps taken to design the MPC controller:
- Estimation or Measurement of system state
- Definition of the Reference and calculation of error
- MPC calculates the actuator commands($u_0$) for N steps in future to minimize the cost function 
$J = \sum_{i=0}^N f(x_{t+i},r_{t+i},u_{t+i})$.
- Actuator command $u_0$ is applied for the current cycle and the whole process is repeated again for next cycle.

![MPC](./Pictures/MPC-vis.jpg)



The designing of MPC for this project involves designing the following

- System Model
- Calculation of Cross Track Error and Heading Error
- Sample Time(dt) and No. of steps in prediction horizon(N)
- Cost Function
- Adapting  the system for actuator delay


## Vehicle Model 

There are two ways to describe the lateral dynamics of a car such as kinematic models, dynamic models. The kinematic models consider only the geometry of the vehicle motion and the car to decribe the behavior. It is simple, less computation intensive and has lesser accuracy compared to dynamic model. The dynamic models considers not only the geometry but also the forces acting on the vehicle to describe the vehicle motion. In this project only a non-linear kinematic model is used to describe the motion of the vehicle. Kinematic model is used because it is less computation intensive and it is has sufficient accuracy to describe the motion of a car in the simulator.The bicycle motion model discussed in the lecture is used in the project. The following picture shows the bicycle model of a car.

![MPC](./Pictures/SingleTrackModel.png)

The following quatities are considered as state variables:
- Position of the car in X-Axis ($x_t$)
- Position of the car in Y-Axis ($y_t$)
- Orientation of the car along Z-Axis ($\psi_t$)
- Magnitude of vehicle velocity ($v_t$)
- Cross Track Error($cte_t$)- minimum distance between actual car position and reference trajectory
- Heading Error($e\psi_t$) - difference between refenrece heading and the actual heading

In this model the error values $cte_t$ and $e\psi_t$ are also considered as state variable because the MPC uses the error values to calculate the cost and minimizes them.

The following equations describe the motion of the car in 2D plane. It has to be noted that the side slip angle i.e. the angle between the velocity vector and the orientation of the car is zero.

$ x_{t+1}    = x_t + v_t cos(\psi_t)dt $

$ y_{t+1}    = y_t + v_t sin(\psi_t)dt $

$ \psi_{t+1} = \psi_t + \frac{v_t}{L_f}{\delta_t}dt$

$ v_{t+1} = v_t + a_tk_adt$

$ cte_{t+1} = f(x_t) - y_t + v_tsin(e\psi_t)dt$

$ e\psi_t+1 = {\psi}des_t + \frac{v_t}{L_f}{\delta_t}dt$

where:
- $dt$ is the sample time
- $\delta_t$ is the steering angle
- $L_f$ is the distance between front wheel and car's center of gravity, which is given in the lecture
- $a_t$ is the throttle position
- $k_a$ is the gain between the throttle position and the acceleration. This value is chosen based on the experiment in the simulator
- $f(x)$ is the curve representing the reference trajectory and $f(x_t)$ is the y coordinate in the reference trajectory for the current x-coordinate of the car
- ${\psi}des_t$ is the desired heading which is calculated from the slope of the reference trajectory.

## Calculation of Cross Track and Heading Error

The MPC contorller requires the cross track and heading error. They are used to calculate the cost function of the MPC which is minimized. The simlulator gives the reference points to the program which are the point lying in the centre of the road on the track. These reference points are used to fit a curve forming a reference trajectory. The error $cte_t$ is the minimum distance between the reference trajectory and the current location of the car. The error $e\psi_t$ is the difference between the slope of the reference trajectory at the point where $cte_t$ is calculated and the current heading of the car. 
The following picture shows a visual representation of the errors.

![Pic](./Pictures/ErrorsVis.png)

The reference points given by the simulator are in global co-ordinate system, it is easier to calculate the errors if the refrence points are transformed to vehicle's co-ordinate system. This transformation can be done in two steps translating the points from global co-ordinates to vehicle centric co-ordinate system but parallel to global co-ordinates. The second step is to rotate the co-ordinate system to compensate the heading of the vehicle. 

![Pic](./Pictures/BodyCentricCoordinates.png)

The different co-ordinate system are shown in the picture above.
- $X,Y$ axis global co-ordinates
- $X_p,Y_p$ vehicle's centric parallel co-ordinate system
- $X_b,Y_b$ vehicle's body centric co-ordinate system

The translation of a point from global co-ordinate system to vehicle's parallel co-ordinate system can be done using the formula

$\vec{r_p} = \vec{r} - \vec{r_{car}}$
where:
 - $\vec{r}$ is the location of an arbitary point in global co-ordinates
 - $\vec{r_p}$ is the location of the point in vehicle's parallel co-ordinates
 - $\vec{r_{car}}$ is the location of the vehicle in global co-ordinates
 
Rotation from vehicle's parallel co-ordinates to vehicle's body centric co-ordinates can be done using formula

$\vec{r_p} = S_{\psi}.\vec{r_b}$
=> $\vec{r_b} = S_{\psi}^{-1}.\vec{r_p}$

where:
- $\vec{r_b}$ is the location of the point in vehicle's body centric co-ordinates
- $S_{\psi}$ is the rotation matrix with respect to Z-axis for a heading angle of $\psi$

The rotation matrix is given by

$S_{\psi} = \begin{bmatrix} cos(\psi) & -sin(\psi) \\  sin(\psi) & cos(\psi)\end{bmatrix}$

$S_{\psi}^{-1} = \begin{bmatrix} cos(\psi) & sin(\psi) \\  -sin(\psi) & cos(\psi)\end{bmatrix}$

Once the points are transformed to body centric co-ordinates a curve is fit using the polyfit() function which returns the coefficients of the reference trajectory $f(x)$. It has to be noted here that the position and heading of the car is zero since the all the points are converted to car's body centric co-ordinate system. The function $f(x_t)$ gives the Y- coordinates of the refrence trajectory for any arbitary x-coordinate and difference between the y-coordinates in the reference trajectory and the y-coordinates of the car($x_t, y_t = 0$ for the car) at time $t$ gives the $cte_t$. The heading of the refrence trajectory $\psi{des}_t$ is the slope of the reference trajectory at $x_t$ and the diffrence between the heading of the reference trajectory $\psi{des}_t$ and the current heading of the car $\psi_t$ ($\psi_t = 0$ ) gives the heading error $e\psi_t$.

$cte_t = f(x_t)- y_t$

$e\psi_t = \psi{des_t} - \psi_t$ 

$ \psi{des_t} = f'(x_t)$ 


## Sample Time(dt) and Prediction Horizon N

The MPC controller optimizes the actuator commands for control horizon ($u_t$ to $u_{t+N-1}$) N-1 steps by predicting the system state ($ x_t$ to $x_{t+N} $) for the prediction horizon N by minimizing the cost function $J = \sum_{i=0}^N f(x_{t+i},r_{t+i},u_{t+i})$. Where N is the prediction horizon the number of steps in the future which the MPC controller optimizes. The sample time $dt$ is the sample time the MPC uses in the model to perform the integration and the derivation in the model prediction. It has to be noted that this sample time of the model is different from the from the sample time of the controller. The cycle time of the controller is usually determined by the specification of the micro-controller, sensors and actuators used in the system. The sample time discussed here is only the sampling time used by the MPC to predict the states of the system using the discrete time state space model used to describe the behavior of the system. Both of the parameters sample time $dt$ and the prediction horizon N has a huge influence in the performance of the MPC controller and so they have to be chosen carefully.

The sampling time $dt$ determines the accuracy of the model as it detmines the accuracy of the integrative and derivative operations used the model. For the model to be accurate the sampling time $dt$ has to be as small as possible. But reducing the sampling time reduces the total time in future the MPC optimizes. The product of sampling time and the prediction horizon determines the total time ($t = dt*N$) in future in which the system response is optimized by the MPC. Increasing the prediction horizon increases the total time optimized by MPC but increasing the N increases the computation effort of the controller as N determines the size of the optimizers solution. Therefore chosing the sample time and the prediction horizon N is a trade off between the performance and feasibility( limited by computation effort).

After several trials following values were chosen:
- $N = 12$
- $dt = 0.1$

## Cost Function

The aim of MPC controller is to minimize the cost function. Choosing the right cost function is required to make the system behave as expected. The primary variables which should be considered in the cost functions are the errors $cte_t$ and $e\psi_t$. All the variables in the cost function are squared in order to get only the absolute value and sum is taken to calculate the total cost.

The following variables are used in the cost function.

|Variable|weight|Purpose|
|-|-|
|Cross track error $cte_t$|800| To keep the vehicle in the center of the road|
|Heading error $e\psi_t$ |800| To  keep the vehicle in the center of the road|
|Difference between reference speed and actual speed $v_{ref}-v$|2| To prevent the vehicle from stopping and to maintain constant speed|
|Steering angle $\delta_t$|100| To reduce the use of steering wheel when the vehicle is not turning|
|Throttle $a_t$|1| To minimize the use of Throttle|
|Change of steering angle $\delta_{t+1} - \delta_t$|100| To minimize the oscillation of the car due to rapid steering|
|Change of throttle $a_{t+1}-a_t$|50|To minimize repid accelaration and deceleration|
|Lateral Acceleration - $a_y = \frac{v^2}{r}$|1| To reduce the speed in the corners which improves safety and comfort|

## Adapting the MPC for time delay

In the real car there is always a finite delay between the actuator command and the actuation. This happens due to the latency of the signal transmission and the dynamics of the actuator. This can be explicitly adapted in the MPC controller. The time delay is simulated in this project by delaying the message to the simulator.There are two ways to adapt the MPC for latency either by calculating the system state for the period of time delay using the model before the signal being sent to the MPC controller or by modeling the time delay in the model used by MPC. In this implementation the a time delay of 100 ms is modeled in the MPC optimizer where the actuator command is delayed for one sample (as $dt = 0.1 s$). Since the delay is modeled in the optimizer it takes care of it acutomatically. Compared to the system without delay the system with time delay of 100 ms required 2 more samples in the prediction horizon to acheive comparable performance. The approach of simulating the system state for a period of time delay was not working because the model used is not very accurate.
